## Word2Vec 을 이용한 평점 및 메타데이터 기반 영화 추천기법

먼저 여러가지 영화 데이터를 로드합니다. <br>
영화 데이터는 MovieLens의 데이터를 사용하였습니다.

In [62]:
import pandas as pd
# 판다스를 import 합니다.

rating = pd.read_csv('ratings.csv')
tag = pd.read_csv('tags.csv')
movie = pd.read_csv('movies.csv')
# 각종 데이터를 로드합니다.

rating_df = pd.DataFrame(rating)
tag_df = pd.DataFrame(tag)
movie_df = pd.DataFrame(movie)
# 데이터 프레임을 만듭니다.

In [63]:
rating_df.head(10)
# 레이팅 데이터

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [64]:
tag_df.head(10)
# 태그 데이터

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [65]:
movie_df.head(10)
# 영화 데이터

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


본 논문에서 제안한 방식은 메타데이터 + 메타데이터 + ... + 메타데이터 + 영화이름으로 문장을 만들고 <br>
각 문장들을 하나의 벡터로 Concatenate 한 뒤에 Word2Vec에 넣어 임베딩하는 것이 첫 순서였습니다.

In [66]:
# 일단 tag 데이터프레임에서 당장에 필요없는 userId 열과 timestamp 열을 제거합니다.
tag_df = tag_df.drop('userId', 1)
tag_df = tag_df.drop('timestamp', 1)

In [67]:
tag_df.head(10)

,movieId,tag
0,60756,funny
1,60756,Highly quotable
2,60756,will ferrell
3,89774,Boxing story
4,89774,MMA
5,89774,Tom Hardy
6,106782,drugs
7,106782,Leonardo DiCaprio
8,106782,Martin Scorsese
9,48516,way too long


이제 각 태그들에 고유한 번호를 매겨야 합니다. <br>
우선 사용자에 관계없이 영화 : 태그로 볼 수 있게 movieId를 기점으로 정렬합니다.


In [69]:
tag_df = tag_df.sort_values(by='movieId')


In [73]:
for movieId, tag in tag_df.values:
    if movieId < 20:
        print(movieId, tag)
    else:
        break


1 fun
1 pixar
1 pixar
2 Robin Williams
2 magic board game
2 fantasy
2 game
3 old
3 moldy
5 remake
5 pregnancy
7 remake
11 president
11 politics
14 politics
14 president
16 Mafia
17 Jane Austen


In [126]:
my_array = []
tmp_array = []
tmp_for_movie_id = 1

for movieId, tag in tag_df.values:

    if tmp_for_movie_id == movieId:
        tmp_array.append(tag)

    else:
        my_array.append(list(tmp_array))
        tmp_for_movie_id = movieId
        tmp_array.clear()
        tmp_array.append(tag)

for i in my_array:
    print(i, '\n')

['fun', 'pixar', 'pixar'] 

['Robin Williams', 'magic board game', 'fantasy', 'game'] 

['old', 'moldy'] 

['remake', 'pregnancy'] 

['remake'] 

['president', 'politics'] 

['politics', 'president'] 

['Mafia'] 

['Jane Austen'] 

['Hollywood'] 

['serial killer'] 

['alcoholism'] 

['Shakespeare'] 

['Jane Austen', 'In Netflix queue'] 

['kidnapping'] 

['teacher', 'high school'] 

['Brad Pitt', 'Bruce Willis', 'mindfuck', 'twist ending', 'time travel', 'Post apocalyptic', 'remake', 'post-apocalyptic', 'time travel', 'time travel'] 

['Animal movie', 'pigs', 'villain nonexistent or not needed for good story'] 

['death penalty', 'Nun'] 

['twins'] 

['Emma', 'Jane Austen', 'seen more than once', 'quotable', 'Paul Rudd', 'funny', 'chick flick'] 

['South Africa', 'In Netflix queue'] 

['Shakespeare'] 

['England'] 

['Journalism'] 

['wedding'] 

['serial killer', 'mystery', 'twist ending'] 

['mindfuck', 'thriller', 'suspense', 'twist ending', 'heist', 'tricky'] 

['adoption', 'prost

 

['disability'] 

['George Bernard Shaw'] 

['E.M. Forster'] 

['In Netflix queue'] 

['martial arts'] 

['Marx brothers'] 

['Hepburn and Tracy'] 

['opera', 'Marx brothers'] 

['immigrants'] 

['In Netflix queue'] 

['surreal', 'Tim Burton', 'sentimental', 'dreamlike'] 

['fantasy world', 'Tolkein', 'great soundtrack', 'lord of the rings', 'scenic', 'stylized', 'Tolkien', 'Adventure', 'ensemble cast', 'fantasy'] 

['Vietnam'] 

['real estate'] 

['serial killer'] 

['Civil War'] 

['Charlotte Bronte'] 

['cross dressing', 'men in drag'] 

['amnesia'] 

['In Netflix queue'] 

['truckers'] 

['circus'] 

['time travel', 'science fiction', 'alternate reality', 'sci-fi'] 

['mountain climbing'] 

['adolescence'] 

['priest', 'religion'] 

['religion'] 

['Cold War'] 

['memory', 'love', 'surreal', 'arthouse', 'artistic', 'atmospheric', 'beautiful', 'bittersweet', 'colourful', 'comedy', 'dreamlike', 'emotional', 'feel-good', 'happpiness', 'humane', 'insightful', 'intelligent', 'lovely',